In [2]:
import sys
!{sys.executable} -m pip install requests pandas lxml html5lib bs4 tweepy
import tweepy
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

In [12]:
def Twitter_API(API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET, filter_tweets, since_date, until_date, total_tweets):
    auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    api = tweepy.API(auth, wait_on_rate_limit=True)

    search_tw = tweepy.Cursor(api.search, q=filter_tweets, since=since_date, until=until_date).items(total_tweets)
    
    tweets_chapo = [
        [tweets.user.screen_name, tweets.user.location, tweets.text, str(tweets.created_at), tweets.user.favourites_count] 
        for tweets in search_tw
    ]

    tweets_chapo = pd.DataFrame(data=tweets_chapo, 
                        columns=['User', "Location", "Tweet", "Date/Time", "Likes"])
    tweets_chapo = tweets_chapo.sort_values(by='Likes', ascending=False)
    tweets_chapo.to_csv('TweetsChapo.csv', index=False)
    
Twitter_API('***',
            '***',
            '***',
            '***', 
            'chapo -filter:retweets', 
            '2019-07-17',
            '2019-07-19',
             5000)

In [9]:
def Project_WebScrapping(url):
    get_data_narco = requests.get(url).content
    get_data_narco = BeautifulSoup(get_data_narco, 'lxml')

    data_narco = get_data_narco.find_all('table', {'class':'wikitable sortable'})

    rows_narco1 = [row.find_all('tr') for row in data_narco]

    rows_narco2 = [r.text.strip().split("\n") for row in rows_narco1 for r in row]

    colnames = rows_narco2[0]
    data_narcos = rows_narco2[1:]

    narcos_df = pd.DataFrame(data_narcos, columns=colnames)
    narcos_df.to_csv('MexicanNarcos.csv', index=False)
    
Project_WebScrapping('https://en.wikipedia.org/wiki/List_of_Mexico%27s_37_most-wanted_drug_lords')